## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-09-19-49-00 +0000,wsj,The Trump administration on Tuesday released i...,https://www.wsj.com/politics/policy/takeaways-...
1,2025-09-09-19-48-13 +0000,bbc,Hamas claims leadership survived Israeli attac...,https://www.bbc.com/news/articles/ced58zywdwno...
2,2025-09-09-19-46-57 +0000,cbc,"Easing Chinese EV tariffs on the table, federa...",https://www.cbc.ca/news/canada/manitoba/china-...
3,2025-09-09-19-45-46 +0000,nyt,Live Updates: Nepali Troops Move to Restore Or...,https://www.nytimes.com/live/2025/09/09/world/...
4,2025-09-09-19-44-53 +0000,nyt,Macron Has No Good Options After Repeat Collap...,https://www.nytimes.com/2025/09/09/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,38
191,new,29
107,nepal,15
202,media,13
200,protests,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
146,2025-09-09-13-31-30 +0000,cbc,Trump says Doha strike on Hamas 'does not adva...,https://www.cbc.ca/9.6895308?cmp=rss,97
59,2025-09-09-17-44-10 +0000,nypost,White House fumes over Epstein birthday book s...,https://nypost.com/2025/09/09/us-news/wh-fumes...,91
29,2025-09-09-18-54-02 +0000,cbc,Nepal's PM resigns after violent protests over...,https://www.cbc.ca/news/world/nepal-protests-g...,87
334,2025-09-08-20-12-00 +0000,nyt,Trump Issues Warning to Hamas as Israeli Milit...,https://www.nytimes.com/2025/09/08/world/middl...,87
203,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,86


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
146,97,2025-09-09-13-31-30 +0000,cbc,Trump says Doha strike on Hamas 'does not adva...,https://www.cbc.ca/9.6895308?cmp=rss
29,87,2025-09-09-18-54-02 +0000,cbc,Nepal's PM resigns after violent protests over...,https://www.cbc.ca/news/world/nepal-protests-g...
226,83,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...
163,53,2025-09-09-12-07-44 +0000,nypost,Zohran Mamdani would still beat Andrew Cuomo i...,https://nypost.com/2025/09/09/us-news/zohran-m...
11,53,2025-09-09-19-29-47 +0000,nypost,Sicko accused of brutally stabbing Ukrainian r...,https://nypost.com/2025/09/09/us-news/ukrainia...
203,48,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...
168,46,2025-09-09-11-48-49 +0000,bbc,Israeli military orders all Gaza City resident...,https://www.bbc.com/news/articles/cvg47kvld8go...
36,45,2025-09-09-18-34-15 +0000,nyt,Macron Picks Sébastien Lecornu as France’s Nex...,https://www.nytimes.com/2025/09/09/world/europ...
286,44,2025-09-08-22-53-00 +0000,nypost,Lachlan Murdoch takes control of media empire ...,https://nypost.com/2025/09/08/media/lachlan-mu...
115,38,2025-09-09-15-03-55 +0000,nyt,"Russian Bomb Hits Crowd of Retirees, Killing a...",https://www.nytimes.com/2025/09/09/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
